# NFL Fantasy Football Projection Model Using XGBoost

This notebook presents a comprehensive workflow for building an NFL fantasy football projection model using XGBoost. Data sources include Yahoo, Pro-Football Reference, and SportsDataIO. The goal is to leverage advanced machine learning techniques and rich datasets to generate accurate player projections for fantasy football analysis and decision-making.

This notebook will use weekly data from the 2024 NFL Regular Season. 

In [2]:
import pandas as pd
import numpy as np
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
print(f"Project Root: {project_root}")
print("Sys Path Before:", sys.path)
if project_root not in sys.path:
    print("Inserting project root to sys.path")
    sys.path.insert(0, project_root)

# Now import
from data_api import SportsDataIO
from data_api import Yahoo
from utils import describe_endpoint
from yahoo_helpers import get_all_players, get_player_details, get_player_stats

from dotenv import load_dotenv
load_dotenv()

Project Root: c:\Users\bengu\Documents\Sports Analysis Project\clairvoyent-raven-sports-analysis\src
Sys Path Before: ['c:\\Users\\bengu\\Documents\\Sports Analysis Project\\clairvoyent-raven-sports-analysis\\src', 'C:\\Users\\bengu\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip', 'C:\\Users\\bengu\\AppData\\Local\\Programs\\Python\\Python310\\DLLs', 'C:\\Users\\bengu\\AppData\\Local\\Programs\\Python\\Python310\\lib', 'C:\\Users\\bengu\\AppData\\Local\\Programs\\Python\\Python310', 'c:\\Users\\bengu\\.virtualenvs\\cfeproj-oIABPDjj', '', 'c:\\Users\\bengu\\.virtualenvs\\cfeproj-oIABPDjj\\lib\\site-packages', 'c:\\Users\\bengu\\.virtualenvs\\cfeproj-oIABPDjj\\lib\\site-packages\\win32', 'c:\\Users\\bengu\\.virtualenvs\\cfeproj-oIABPDjj\\lib\\site-packages\\win32\\lib', 'c:\\Users\\bengu\\.virtualenvs\\cfeproj-oIABPDjj\\lib\\site-packages\\Pythonwin']


True

In [3]:
# Initialize API wrappers
sdi_api = SportsDataIO(api_key=os.getenv("SPORTS_DATA_IO_API_KEY"))
yahoo_api = Yahoo(os.getenv("YAHOO_OAUTH_KEYS_PATH"))

[2025-09-26 20:01:17,459 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2025-09-26 20:01:17,467 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 15729.07385635376
[2025-09-26 20:01:17,468 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN HAS EXPIRED
[2025-09-26 20:01:17,469 DEBUG] [yahoo_oauth.oauth.refresh_access_token] REFRESHING TOKEN


Getting league key
League key: 461.l.242497


## Extract and Clean Necessary Data

Needed data

* 2024 Regular Season Weekly Stats (Sports Data IO)
* 2023 Regular Season Yearly Stats (Yahoo Fantasy)


In [19]:
all_players = get_all_players()
print(f"Total players loaded: {len(all_players)}")

# Get details for a sample of players
sample_player_details = get_player_details(yahoo_api, all_players[:5])

player_ids = []
for player in sample_player_details:
    player_ids.append(player["player_id"])

sample_player_stats = get_player_stats(yahoo_api, [player_id for player_id in player_ids], req_type="season", season=2023)

Total players loaded: 2158
Error fetching details for A'Shawn Robinson: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


In [16]:
player_game_stats_2024 = sdi_api.fantasy.get_player_game_stats(season="2024REG", week=1)